### Курсовой проект
В проектной работе хотел бы сделать анализ по годам выпуска фильма по данным imdb.com.<br>
По годам выпуска рассчитать основные статистические показатели: Мода, медиана, квартили, дисперсию.<br>
Проверить, работает ли нормальное распределение и Паретто.<br>
Проверить есть ли зависимость бюджетов, рейтингов, жанров от года выпуска фильмов.<br>
Посмотреть поведение год выпуска во времени.<br>
В идеале, хотел бы сделать прогноз на 2020 год: какие жанры с какой вероятностью выйдут определенное количество фильмов с определенным бюджетом и из какой страны<br>

In [3]:
import pandas as pd
import numpy as np

In [24]:
akas = pd.read_csv('data_akas.csv', sep='\t', low_memory=False)
akas.dtypes

titleId            object
ordering            int64
title              object
region             object
language           object
types              object
attributes         object
isOriginalTitle    object
dtype: object

In [26]:
akas_f = akas[akas['isOriginalTitle'] == '1']
akas_f

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
4,tt0000001,5,Carmencita,\N,\N,original,\N,1
5,tt0000002,1,Le clown et ses chiens,\N,\N,original,\N,1
14,tt0000003,4,Pauvre Pierrot,\N,\N,original,\N,1
17,tt0000004,1,Un bon bock,\N,\N,original,\N,1
30,tt0000005,8,Blacksmith Scene,\N,\N,original,\N,1
36,tt0000006,5,Chinese Opium Den,\N,\N,original,\N,1
38,tt0000007,2,Corbett and Courtney Before the Kinetograph,\N,\N,original,\N,1
47,tt0000008,4,Edison Kinetoscopic Record of a Sneeze,\N,\N,original,\N,1
50,tt0000009,1,Miss Jerry,\N,\N,original,\N,1
54,tt0000010,11,La sortie de l'usine Lumière à Lyon,\N,\N,original,\N,1


In [8]:
ratings = pd.read_csv('data_ratings.csv', sep='\t', low_memory=False)
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.6,1552
1,tt0000002,6.1,186
2,tt0000003,6.5,1212
3,tt0000004,6.2,113
4,tt0000005,6.1,1939
5,tt0000006,5.2,103
6,tt0000007,5.4,617
7,tt0000008,5.4,1671
8,tt0000009,5.4,81
9,tt0000010,6.9,5565


In [7]:
basics = pd.read_csv('data_basics.csv', sep='\t', low_memory=False)
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
9,tt0000010,short,Exiting the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"
